# Analisis de Sentimiento (simple)

El objetivo de este ejemplo es hacer el Analisis de Sentimeinto de reseñas de películas, viendo si es positivo o negativo.

Vamos a utilizar la libreria NLTK, asi como otras herramientas.

- De **NLTK** usaremos el *Lemmatizer* que aplica lematización a las palabras. Simplificando mucho, un lematizador convierte las palabras a su forma básica, por ejemplo las palabras "perro", "perros" denotan lo mismo, asi que solo deja la raiz de la palabra. Esto tiene el efecto de reducir el tamaño del diccionario.

- Tambien usaremos **BeautifulSoup** porque los datos vienen en XML y necesitamos extraerlos.

- Por ultimo, para la clasificación usaremos el típico clasificador de **Regresión Logistica** de sklearn. La clasifiación de sentimientos es binaria, positivo o negativo.

In [1]:
import nltk
import numpy as np

from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from bs4 import BeautifulSoup

In [2]:
# Instanciamos un objeto de la claseWordNetLemmatizer()
wordnet_lemmatizer = WordNetLemmatizer()



In [3]:
# Cargamos las 'stopwords' del idioma Ingles de algun sitio de internet
# from http://www.lextek.com/manuals/onix/stopwords1.html
stopwords = set(w.rstrip() for w in open('stopwords.txt'))

### Tenemos 3 archivos con las reseñas o criticas. 2 de ellos están etiquetados como POSITIVAS o NEGATIVAS. 

### El 3ro está sin etiquetar.

In [18]:
# Cargamos las criticas POSITIVAS
# data courtesy of http://www.cs.jhu.edu/~mdredze/datasets/sentiment/index2.html
positive_reviews = BeautifulSoup(open('/home/jorge/data/sorted_data_acl/electronics/positive.review').read())


In [19]:
# Dejamos solo las del campo 'review_text'
positive_reviews = positive_reviews.findAll('review_text')

In [20]:
# por ejemplo la 10 reseña positiva sería:
positive_reviews[10]

<review_text>\nI am very happy with this product. It folds super slim, so traveling with it is a breeze! Pretty good sound - not Bose quality, but for the price, very respectable! I've had it almost a year, and it has been along on many weekend get-aways, and works great. I use it alot, so it was a good purchase for me\n</review_text>

In [21]:
# Hacemos lo mismo con las Negativas
negative_reviews = BeautifulSoup(open('/home/jorge/data/sorted_data_acl/electronics/negative.review').read())
negative_reviews = negative_reviews.findAll('review_text')

In [22]:
negative_reviews[3]

<review_text>\nI knew these were inexpensive CD cases, but I can't even open one without it breaking into two pieces..\n</review_text>

In [23]:
# Tenemos muchas más revisiones POSITIVAS que NEGATIVAS, asi que tenemos un problema de balance

# Tomaremos una muestra de las positivas del mismo tamaño que las negativas, 
# asi tenemos las clases balanceadas
positive_reviews = positive_reviews[:len(negative_reviews)]


## Creación del Diccionario

El diccionario debe incluir todas las palabras de nuestro vocabulario. 
Para ello tenemos que centrarnos en las 'palabras' que son representativas y utiles. Por lo tanto necesitamos tokenizar (eliminando distinciones entre mayusculas y minusculas) y luego lemmatizar para dejar las palabras en su forma basica.



### Primero vamos a tratar de tokenizar los textos usando el tokenizador de NLTK.

In [26]:
# A ver como tokeniza la 2da critica
t = positive_reviews[2]
nltk.tokenize.word_tokenize(t.text)

[u'Wish',
 u'the',
 u'unit',
 u'had',
 u'a',
 u'separate',
 u'online/offline',
 u'light',
 u'.',
 u'When',
 u'power',
 u'to',
 u'the',
 u'unit',
 u'is',
 u'missing',
 u',',
 u'the',
 u'single',
 u'red',
 u'light',
 u'turns',
 u'off',
 u'only',
 u'when',
 u'the',
 u'warning',
 u'sounds',
 u'.',
 u'The',
 u'warning',
 u'sound',
 u'is',
 u'like',
 u'a',
 u'lot',
 u'of',
 u'sounds',
 u'you',
 u'hear',
 u'in',
 u'the',
 u'house',
 u'so',
 u'it',
 u'is',
 u"n't",
 u'always',
 u'easy',
 u'to',
 u'tell',
 u'what',
 u'is',
 u'happening']

Vemos que la tokenización está considerando distintas las palabras minusculas y mayusculas.

Vamos a crear una función para tokenizar mejor.

In [27]:
def my_tokenizer(s):
    s = s.lower() # downcase
    tokens = nltk.tokenize.word_tokenize(s) # separamos el string en "palabras" (tokens)
    tokens = [t for t in tokens if len(t) > 2] # eliminamos las palabras cortas, probablemente no son utiles
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] # Lematizamos --> palabras en forma base
    tokens = [t for t in tokens if t not in stopwords] # quitamos stopwords
    return tokens

### Ahora si podemos crear el diccionario...

In [28]:
# Creamos un mapa word-to-index de tal manera que podamos crear nuestros vectores de frecuencias mas tarde
# El diccionario es un conjuto de pares K:V donde K es la palabra y V el indice

# Tambien guardamos la tokenizacion en un par de listas para no tenerla que hacer mas adelante.

# Reseñas POSITIVAS
word_index_map = {}  # Creamos un diccionario vacio
current_index = 0
positive_tokenized = []
negative_tokenized = [] # Creamos un par de listas vacias

for review in positive_reviews:         # recorremos las reseñas POSITIVAS
    tokens = my_tokenizer(review.text)       # tokenizamos la reseña
    positive_tokenized.append(tokens)        # metemos la reseña tokenizada a la lista correspondiente
    
    for token in tokens:                     # por cada token de la reseña
        if token not in word_index_map:            # verifico si NO esta en el diccionario
            word_index_map[token] = current_index       # entonces lo incluyo la palabra en el dicc y su indice
            current_index += 1                          # e incremento el contador

In [29]:
# Hago lo mismo pero para las NEGATIVAS, pero en el mismo diccionario
for review in negative_reviews:
    tokens = my_tokenizer(review.text)
    negative_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

In [35]:
print "Nuestro diccionario tiene:", len(word_index_map), "palabras."

Nuestro diccionario tiene: 11088 palabras.


## Creación de las matrices de Entrada

In [36]:
# Creamos una función que transforma tokens en vectores

def tokens_to_vector(tokens, label):

    x = np.zeros(len(word_index_map) + 1) # creamos un vector de nulos del tamaño del diccionario
                                          # y agregamos 1 para la etiqueta
    
    for t in tokens:                      # recorremos los tokens de entrada
        i = word_index_map[t]                  # guardo el indice de la palabra actual en i
        x[i] += 1                              # incremento el contador del vector x[] en 1

    # Al final del proceso, me queda un vector con el numero de repeticiones de cada palabra en la posición que 
    # tiene en el diccionario.

    # Ahora normalizamos de tal manera que todo sume 1 
    x = x / x.sum()                       # normalizamos antes de asignar la etiqueta
    x[-1] = label
    return x

In [37]:
# Cual es el tamaño de nuestros datos de entrada??

# es el numero de palabras tokenizadas de las reseñas positivas y negativas, entonces:

N = len(positive_tokenized) + len(negative_tokenized)

N

2000

In [38]:


# (N x D+1 matrix - keeping them together for now so we can shuffle more easily later
data = np.zeros((N, len(word_index_map) + 1))  ## N x D+1 (mio)
i = 0
for tokens in positive_tokenized:
    xy = tokens_to_vector(tokens, 1)
    data[i,:] = xy
    i += 1



In [39]:
data

array([[ 0.02272727,  0.06818182,  0.02272727, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.        ,  0.        ,  0.08333333, ...,  0.        ,
         0.        ,  1.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [40]:
for tokens in negative_tokenized:
    xy = tokens_to_vector(tokens, 0)
    data[i,:] = xy
    i += 1

In [46]:
data[3,:]

array([ 0.,  0.,  0., ...,  0.,  0.,  1.])

In [47]:
# Esta bien que de 2 porque suma 1 los datos normalizados + 1 de la etiqueta
data[3,:].sum()

2.0

## Entrenamos con los datos que hemos preparado

In [53]:
# Hacemos shuffle de los datos y creamos datos de train/test 

np.random.shuffle(data)

X = data[:,:-1]
Y = data[:,-1]


In [49]:
# Las ultimas 100 seran de test
Xtrain = X[:-100,]
Ytrain = Y[:-100,]
Xtest = X[-100:,]
Ytest = Y[-100:,]

In [51]:
model = LogisticRegression()
model.fit(Xtrain, Ytrain)
print "Scoring de clasificacion:", model.score(Xtest, Ytest)


Scoring de clasificacion: 0.79


In [52]:
#Veamos los pesos de cada palabra

threshold = 0.7
for word, index in word_index_map.iteritems():
    weight = model.coef_[0][index]
    if weight > threshold or weight < -threshold:
        print word, weight

easy 1.7934926583
time -0.807642920518
love 1.17577946379
returned -0.72663482898
perfect 0.951307505762
waste -0.927823778335
highly 0.994149449428
wa -1.61891731543
support -0.848011306122
price 2.60785214634
lot 0.71252030231
you 1.09301794223
poor -0.781009226709
month -0.874442029631
tried -0.805179215599
pretty 0.707550889108
quality 1.34952535326
speaker 0.820884704347
ha 0.80513032191
recommend 0.729065468829
doe -1.16068368505
bad -0.815918568832
item -0.983779411579
little 0.946090010437
sound 0.985615740155
n't -2.0375472747
then -1.18894130318
money -1.09925261759
've 0.858329963811
buy -0.784170142213
excellent 1.38226410504
memory 0.964715412386
week -0.752582434415
return -1.13320762448
fast 0.817628195731
